In [1]:
####Package Imports######
from skimage.metrics import structural_similarity
import imutils
import cv2
import os

In [5]:
import anvil.media
import numpy as np
import csv


@anvil.server.callable



def compare_image(file1, file2):
    
  
        image1 = cv2.imread(FN1)
        image2 = cv2.imread(FN2)
            
            
        dim1 = image1.shape
        dim2 = image2.shape
        chk_dim = np.array_equal(dim1,dim2)
        
        ####Normalization###########
        

        
        if (chk_dim is True):
            
            scale_percent = 35

            width1 = int(image1.shape[1] * scale_percent / 100)
            height1 = int(image1.shape[0] * scale_percent / 100)

            width2 = int(image2.shape[1] * scale_percent / 100)
            height2 = int(image2.shape[0] * scale_percent / 100)

            dsize1 = (width1, height1)
            dsize2 = (width2, height2)

            R1 = cv2.resize(image1,dsize1)
            R2 = cv2.resize(image2,dsize2)
        
        
    ####convert to grayscale image###########
    
            grayR1 = cv2.cvtColor(R1, cv2.COLOR_BGR2GRAY)
            grayR2 = cv2.cvtColor(R2, cv2.COLOR_BGR2GRAY)

    ######find Similarity Index###########
       

            (score, diff) = structural_similarity(grayR1, grayR2, full=True)

            diff = (diff * 255).astype("uint8")##convert from floating point to unsigned int for openCV
        
            thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
        
            location = 'C:/Users/geral/Documents/SeniorProjectTest/Compare'
       
            
    ##### loop over the contours compute the bounding box draw bounding box on diffrences #####
            
            for c in cnts:
            
                (x, y, w, h) = cv2.boundingRect(c)
                cv2.rectangle(R1, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.rectangle(R2, (x, y), (x + w, y + h), (0, 0, 255), 2)
                coords1 = np.asarray(c)
                coords2 = np.squeeze(coords1, axis = 1)
                
        
            np.savetxt((os.path.join(location , 'Coordinates.csv')),coords2, fmt = '%.2f', delimiter =',')
            
       
       
            cv2.imwrite(os.path.join(location , 'Floorplan1.png'), R1)
            cv2.imwrite(os.path.join(location , 'Floorplan2.png'), R2)
            cv2.imwrite(os.path.join(location , 'Difference.png'), diff)
            cv2.imwrite(os.path.join(location , 'Threshold.png'), thresh)
        
  
            return(float(score))
        
            
        else:
            
            error = 'PLEASE UPLOAD IMAGES WITH SAME DIMENSIONS'
            return(error)

In [7]:
from flask import Flask, request, render_template, flash, redirect, url_for

app = Flask(__name__)

